Тут будет решение 4 лабы и описание теории

здесь применяется метод ортогонализации граммом и шмидтом

Дана СЛАУ всё того же вида Ax = b

Запишем расширенную матрицу:

$\left(
  \begin{array}{cccc|c}
      a_{11} & a_{12} & \dots & a_{1n}  & b_1 \\
      a_{21} & a_{22} & \dots & a_{2n}  & b_2 \\
      \vdots &        & \ddots & \vdots & b_3 \\
      a_{11} & a_{12} & \dots & a_{1n}  & b_n \\
  \end{array}
\right)$

И теперь мы хотим найти решение каким-то ортогональным образом

Тогда дополним расширенную матрицу вектором $\begin{pmatrix} 0  & 0 & \dots & 0 & 1\end{pmatrix}$ до следующего вида:

$\left(
  \begin{array}{cccc|c}
      a_{11} & a_{12} & \dots & a_{1n}  & b_1 \\
      a_{21} & a_{22} & \dots & a_{2n}  & b_2 \\
      \vdots &        & \ddots & \vdots & b_3 \\
      a_{11} & a_{12} & \dots & a_{1n}  & b_n \\
      0      & 0      & \dots & 0       & 1   \\
  \end{array}
\right)$

Теперь каждые строки ортогонализуем методом Грама-Шмидта
и получаем решение Matrix[i, 0:N] / Matrix[N, N]

Надо будет дописать, так как ничего не обосновано

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from typing import Tuple
from sympy import Matrix, init_printing
init_printing()

def isEqual(x, y) -> bool:
  return np.isclose(x, y, rtol=1e-05, atol=1e-08, equal_nan=False)

def compareSolutions(x_1: np.ndarray, x_2: np.ndarray) -> bool:
  for i in range(x.shape[1]):
    if (not isEqual(x_1[i], x_2[i])):
      return False
  return True

def GenerateMatrixOnes(N: int) -> Tuple[np.ndarray, np.ndarray]:
  Matr = np.full((N, N), 0.5) - np.random.rand(N, N)
  A = np.dot(Matr.T, Matr)
  b = np.zeros((N, 1))
  for i in range(N):
    b[i] += np.sum(A[i, :])

  return (A, b)

def GenerateRandomMatrix(N: int) -> Tuple[np.ndarray, np.ndarray]:
  return ( np.random.randint(30, size=(N, N)), np.random.randint(30, size=(N, 1)) )

def ConcatAndAddVec(A, b) -> np.ndarray:
  Mtrx = np.concatenate((A, b * (-1)), axis=1)
  Mtrx = np.concatenate((Mtrx, np.zeros((1, Mtrx.shape[1]))), axis=0)
  Mtrx[N,N] = 1
  return Mtrx

# given (v_1, v_2, ..., v_n)
# need to
# g_1 = v_1
# g_2 = v_2 - a * g_1
# g_3 = v_3 - b * g_1 - c * g_2
# ...
# a = (g_1, v_2) / (g_1, g_1)
#
# b = (g_1, v_3) / (g_1 , g_1)
# c = (g_2, v_3) / (g_2, g_2)

def GrammShmidt(A: np.ndarray, b: np.ndarray) -> np.ndarray:
  Mtrx = ConcatAndAddVec(A, b)
  N = A.shape[0]

  for i in range(N + 1):
    v_i = Mtrx[i]

    g_i = v_i.copy()
    for j in range(i):
      g_j = Mtrx[j]

      coeff = np.dot(v_i, g_j) / np.dot(g_j, g_j)
      g_i -= coeff * Mtrx[j]

    Mtrx[i] = g_i

  for i in range(N + 1):
    Mtrx[i] = Mtrx[i] / np.linalg.norm(Mtrx[i])

  solution = Mtrx[N, 0:N] / Mtrx[N, N]
  return solution.reshape((N, 1))

Применяем полученное знание свыше и заодно проверяем верно ли нашли решение.

Асимптотика аналогична методу Гаусса, раве что константы могут быть различны.

Также было выявлено, что погрешность вычислений накапливается быстрее, чем в методе Гаусса, так что способ вроде бы и хорош, но не без недостатков.

In [ ]:
N = 4
A, b = GenerateRandomMatrix(N)
print("Сгенерированная матрица:")
display(Matrix(A))
print("Сгенерированная правая часть:")
display(Matrix(b))

x = GrammShmidt(A, b)
print("Найденное решение СЛАУ:")
display(Matrix(x))

print("Невязка:")
display(Matrix(np.matmul(A, x) - b))

assert(compareSolutions(np.linalg.solve(A, b), x))

Сгенерированная матрица:


⎡15  26  8   22⎤
⎢              ⎥
⎢1   22  14  22⎥
⎢              ⎥
⎢18  3   1   11⎥
⎢              ⎥
⎣8   18  7   20⎦

Сгенерированная правая часть:


⎡25⎤
⎢  ⎥
⎢20⎥
⎢  ⎥
⎢17⎥
⎢  ⎥
⎣11⎦

Найденное решение СЛАУ:


⎡1.62167300380229 ⎤
⎢                 ⎥
⎢0.306321292775662⎥
⎢                 ⎥
⎢3.15478453738911 ⎥
⎢                 ⎥
⎣-1.4785329531052 ⎦

Невязка:


⎡1.4210854715202e-14⎤
⎢                   ⎥
⎢         0         ⎥
⎢                   ⎥
⎢8.5265128291212e-14⎥
⎢                   ⎥
⎣-7.105427357601e-15⎦